In [271]:
import pandas as pd
import missingno as msno
import re
import numpy as np

In [272]:
df = pd.read_csv("sakam_knigi.csv", encoding='utf-8-sig')

In [273]:
df.head()

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
0,Фаворити,Лејн Фарго,590.0,590.0,False,Романси,2025-05-02
1,Тунел,Ана Волц,400.0,400.0,False,Романси,2025-05-02
2,Љубов во Рим,Сара Адамс,490.0,490.0,False,Романси,2025-05-02
3,Шпанска љубовна измама,Елена Армас,590.0,590.0,False,Романси,2025-05-02
4,Изопачени лаги,Ана Хуанг,590.0,590.0,False,Романси,2025-05-02


### Clean Author Column

In [274]:
df['Author'] = df['Author'].str.replace("од:", "")

In [275]:
df.to_csv("sakam_knigi.csv", encoding='utf-8-sig', index=False)
df = pd.read_csv("sakam_knigi.csv", encoding='utf-8-sig')

In [276]:
df.head()

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
0,Фаворити,Лејн Фарго,590.0,590.0,False,Романси,2025-05-02
1,Тунел,Ана Волц,400.0,400.0,False,Романси,2025-05-02
2,Љубов во Рим,Сара Адамс,490.0,490.0,False,Романси,2025-05-02
3,Шпанска љубовна измама,Елена Армас,590.0,590.0,False,Романси,2025-05-02
4,Изопачени лаги,Ана Хуанг,590.0,590.0,False,Романси,2025-05-02


In [277]:
df.isnull().sum()

Title              0
Author          1592
Real Price         2
Sale Price         2
IsOnSale           0
Category           0
Retrieved At       0
dtype: int64

### Duplicates
- 10 entries are completely identical. We can merge the, Keep only one

In [278]:
duplicates = df[df.duplicated(keep=False)]
duplicates = duplicates.sort_values(by=['Title', 'Author'])
print("Length of duplicates: ", len(duplicates))
duplicates.head(10)

Length of duplicates:  2


,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
975,ИСТОРИЈА НА РЕЛИГИИТЕ,Александар Мењ,800.0,800.0,False,Духовна литература,2025-05-02
976,ИСТОРИЈА НА РЕЛИГИИТЕ,Александар Мењ,800.0,800.0,False,Духовна литература,2025-05-02


In [279]:
print("Original:", len(df))
df = df.drop_duplicates()
print("After drop_duplicates:", len(df))
print("Duplicates removed:", len(df) - len(df))

Original: 2167
After drop_duplicates: 2166
Duplicates removed: 0


### Author

- All 6 Harry Poter books have a missing value in the Author field. Replace all with  'Џ.К. Роулинг'

In [280]:
harry_potter_rows = df[df['Title'].str.contains('Хари Потер', case=False, na=False)]
harry_potter_rows

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
448,Хари Потер и Каменот на мудроста (прв дел),Џ.К. Роулинг,650.0,650.0,False,Мистерии,2025-05-02
449,Хари Потер и и затвореникот од Азкабан,Џ.К. Роулинг,700.0,700.0,False,Мистерии,2025-05-02
450,Хари Потер и Одајата на тајните,Џ.К. Роулинг,650.0,650.0,False,Мистерии,2025-05-02
718,Хари Потер и Редот на фениксот,Џ.К. Роулинг,1150.0,1150.0,False,За млади,2025-05-02
1173,Хари Потер и Редот на фениксот,Џ.К. Роулинг,1150.0,1150.0,False,Трилери,2025-05-02
1572,Хари Потер и Редот на фениксот,Џ.К. Роулинг,1150.0,1150.0,False,Драми,2025-05-02


In [281]:
df.loc[df['Title'].str.contains('Хари Потер', case=False, na=False), 'Author'] = 'Џ.К. Роулинг'

In [282]:
df.to_csv("sakam_knigi.csv", encoding='utf-8-sig', index=False)
df = pd.read_csv("sakam_knigi.csv", encoding='utf-8-sig')

- Extract authors from the titles that contain authors(e.g. "Избрани песни – Ана Ахматова")

In [283]:
# See which authord have – or  - or —
authors_fix = df[df['Title'].str.contains(r'\s[–\-—]\s\w+(?:\s\w+)?', regex=True, case=False)]
authors_fix.head(100)

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
380,Црниот мачор – раскази за мистеријата,NaN,300.0,300.0,False,Мистерии,2025-05-02
539,Атлас – Приказната на Тате Солт,NaN,940.0,940.0,False,Историски,2025-05-02
599,Снеговите на љубовта – книга втора,NaN,199.0,199.0,False,Историски,2025-05-02
600,Снеговите на љубовта – книга прва,NaN,199.0,199.0,False,Историски,2025-05-02
803,Мудар родител – среќен тинејџер,NaN,600.0,600.0,False,Психологија,2025-05-02
833,Мудар родител – време е за градинка,NaN,650.0,650.0,False,Психологија,2025-05-02
863,Умешноста да ја креирате вашата реалност – Вод...,NaN,450.0,450.0,False,Психологија,2025-05-02
885,МУДАР РОДИТЕЛ – СРЕЌНО БЕБЕ,NaN,600.0,600.0,False,Психологија,2025-05-02
887,ПАТОТ КОН ВЕЛИЧЕСТВЕНОСТА – книга 2,NaN,400.0,400.0,False,Психологија,2025-05-02
969,Македонија и Македонците во Библијата – Библис...,NaN,800.0,800.0,False,Духовна литература,2025-05-02


In [284]:
updates = {
   521: {'Title': 'Избрани песни', 'Author': 'Ана Ахматова'},
   770: {'Title': 'Подарокот', 'Author': 'Едит Егер'},
   843: {'Title': 'Херој', 'Author': 'Р.Бирн'},
   976: {'Title': 'ИСТОРИЈА НА РЕЛИГИИТЕ', 'Author': 'Александар Мењ'}
}

for idx, update in updates.items():
    df.at[idx, 'Title'] = update['Title']
    df.at[idx, 'Author'] = update['Author']

In [285]:
# df[df['Title'].str.contains('ИСТОРИЈА НА РЕЛИГИИТЕ', case=False)]
# df[df['Title'].str.contains('Херој', case=False)]
# df[df['Title'].str.contains('Избрани песни', case=False)]
# df[df['Title'].str.contains('Подарокот', case=False)]

In [286]:
df.to_csv("sakam_knigi.csv", encoding='utf-8-sig', index=False)
df = pd.read_csv("sakam_knigi.csv", encoding='utf-8-sig')

- Check for 'и'

In [287]:
two_authors_df = df[df['Author'].str.contains(r'\sи\s', case=False, na=False)]
two_authors_df

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At


In [288]:
two_authors_df = df[df['Author'].str.contains(r'\sи\s', case=False, na=False)].copy()

two_authors_df['Author'] = two_authors_df['Author'].str.split(' и ')

exploded_df = two_authors_df.explode('Author', ignore_index=True)

df = pd.concat([df[~df['Author'].str.contains(r'\sи\s', case=False, na=False)], exploded_df], ignore_index=True)

In [289]:
two_authors_df = df[df['Author'].str.contains(r'\sи\s', case=False, na=False)]
two_authors_df

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At


In [290]:
df.to_csv("sakam_knigi.csv", encoding='utf-8-sig', index=False)
df = pd.read_csv("sakam_knigi.csv", encoding='utf-8-sig')

- Check for Authors divided by , Example: 1060	Аурис	 Себастијан Фицек, Винсент Клиш,

In [291]:
# # Find rows with commas in Author
comma_rows = df['Author'].str.contains(r'\s*,\s*', na=False)

# # Remove trailing commas in those rows
df.loc[comma_rows, 'Author'] = df.loc[comma_rows, 'Author'].str.rstrip(', ')


In [292]:
comma_authors_df = df[df['Author'].str.contains(r'\s*,\s*', na=False)]
comma_authors_df

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At


In [293]:
comma_authors_df = df[df['Author'].str.contains(r'\s*,\s*', na=False)].copy()

comma_authors_df['Author'] = comma_authors_df['Author'].str.rstrip(', ')
comma_authors_df['Author'] = comma_authors_df['Author'].str.split(r'\s*,\s*')
exploded_comma_df = comma_authors_df.explode('Author', ignore_index=True)
df = df[~df['Author'].str.contains(r'\s*,\s*', na=False)]
df = pd.concat([df, exploded_comma_df], ignore_index=True)


In [294]:
comma_authors_df = df[df['Author'].str.contains(r'\s*,\s*', na=False)]
comma_authors_df

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At


- Find titles that appear under multiple authors <br/> The books from before have multiple authors: (Кралицата Шарлот, Да можев да ти го подарам Божиќ, Аурис, Совршената замена)

In [295]:
# Group by Title and collect unique authors
title_authors = df.groupby('Title')['Author'].unique()

# Reset to a DataFrame
title_authors_df = title_authors.reset_index()

# Add a count of how many authors per title
title_authors_df['Author Count'] = title_authors_df['Author'].apply(len)

# Filter titles with more than one author (i.e., different authors for the same book title)
duplicate_titles = title_authors_df[title_authors_df['Author Count'] > 1]

# Sort by number of authors if desired
duplicate_titles = duplicate_titles.sort_values(by='Author Count', ascending=False)

# Show the result
duplicate_titles.head(200)


,Title,Author,Author Count
1132,Подарокот,"[Едит Егер, Себастијан Фицек, nan]",3
260,Вистинскиот,"[ Џон Марс, nan]",2
126,Аурис,"[ Себастијан Фицек, Винсент Клиш]",2
357,Да можев да ти го подарам Божиќ,"[ Џенифер Л. Мајер, Лин Плорд]",2
757,Кралицата Шарлота,"[ Џулија Квин, Шонда Рајмс]",2
1411,Совршената замена,"[ Грир Хендрикс, Сара Пеканен]",2
1592,Херој,"[ Саманта Јанг, Р.Бирн]",2


In [296]:
df.isnull().sum()

Title              0
Author          1591
Real Price         2
Sale Price         2
IsOnSale           0
Category           0
Retrieved At       0
dtype: int64

2. Clean Price
- Remove "ден" from "Real Price" and "Sale Price" cols

In [297]:
missing_prices_df = df[df['Real Price'].isna() | df['Sale Price'].isna()]
missing_prices_df

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
1142,Азбучната куќа,NaN,NaN,NaN,False,Трилери,2025-05-02
1851,Коалата што не можеше да заспие,NaN,NaN,NaN,False,Детски книги,2025-05-03


### Clean price cols
- Before converting to numeric,remove "ден", remove commas and strip whitespace


In [298]:
# # Clean price columns
# df['Real Price'] = df['Real Price'].str.replace("ден", "", regex=False).str.replace(",", "").str.strip()
# df['Sale Price'] = df['Sale Price'].str.replace("ден", "", regex=False).str.replace(",", "").str.strip()

# # Convert to numeric
# df["Real Price"] = pd.to_numeric(df["Real Price"], errors="coerce")
# df["Sale Price"] = pd.to_numeric(df["Sale Price"], errors="coerce")


In [299]:
df.head()

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
0,Фаворити,Лејн Фарго,590.0,590.0,False,Романси,2025-05-02
1,Тунел,Ана Волц,400.0,400.0,False,Романси,2025-05-02
2,Љубов во Рим,Сара Адамс,490.0,490.0,False,Романси,2025-05-02
3,Шпанска љубовна измама,Елена Армас,590.0,590.0,False,Романси,2025-05-02
4,Изопачени лаги,Ана Хуанг,590.0,590.0,False,Романси,2025-05-02


In [300]:
df.isnull().sum()

Title              0
Author          1591
Real Price         2
Sale Price         2
IsOnSale           0
Category           0
Retrieved At       0
dtype: int64

In [301]:
missing_prices_df = df[df['Real Price'].isna() | df['Sale Price'].isna()]
missing_prices_df.head(20)

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
1142,Азбучната куќа,NaN,NaN,NaN,False,Трилери,2025-05-02
1851,Коалата што не можеше да заспие,NaN,NaN,NaN,False,Детски книги,2025-05-03


### Missing values

In [302]:
# msno.heatmap(df)

- The heatmap shows that the two columns have missing values in exactly the same rows.
- If Real Price is NaN → Sale Price is also NaN.
- If Sale Price is NaN → Real Price is also NaN.



### Sale column
- Make Sale a bool field

In [303]:
# df["Sale"] = df["Sale"].astype(bool)

In [304]:
df = df.rename(columns={"Sale": "IsOnSale"})

In [305]:
df.to_csv("sakam_knigi.csv", encoding='utf-8-sig', index=False)
df = pd.read_csv("sakam_knigi.csv", encoding='utf-8-sig')

In [306]:
df.head()

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
0,Фаворити,Лејн Фарго,590.0,590.0,False,Романси,2025-05-02
1,Тунел,Ана Волц,400.0,400.0,False,Романси,2025-05-02
2,Љубов во Рим,Сара Адамс,490.0,490.0,False,Романси,2025-05-02
3,Шпанска љубовна измама,Елена Армас,590.0,590.0,False,Романси,2025-05-02
4,Изопачени лаги,Ана Хуанг,590.0,590.0,False,Романси,2025-05-02


### Retrieved At Column
- Convert Retrieved At to datetime

In [307]:
df['Retrieved At'] = pd.to_datetime(df['Retrieved At'])

In [308]:
df.to_csv("sakam_knigi.csv", encoding='utf-8-sig', index=False)
df = pd.read_csv("sakam_knigi.csv", encoding='utf-8-sig')

### Fix
-  When a book is not on sale, the Sale Price is equal to the Real Price --> Set the Sale Price to 0 to all rows where IsOnSale==False.Except for the rows where

In [309]:
df.IsOnSale.value_counts()

IsOnSale
False    2150
True       16
Name: count, dtype: int64

In [310]:
exclude_indices = df[(df['IsOnSale'] == False) & (df['Sale Price'] != df['Real Price'])].index


In [311]:
df.loc[(df['IsOnSale'] == False) & (~df.index.isin(exclude_indices)), 'Sale Price'] = 0

In [312]:
df.isnull().sum()

Title              0
Author          1591
Real Price         2
Sale Price         2
IsOnSale           0
Category           0
Retrieved At       0
dtype: int64

### Create Discount (%):
- If IsOnSale is True, calculate discount info(discount percentage)

In [313]:
df

,Title,Author,Real Price,Sale Price,IsOnSale,Category,Retrieved At
0,Фаворити,Лејн Фарго,590.0,0.0,False,Романси,2025-05-02
1,Тунел,Ана Волц,400.0,0.0,False,Романси,2025-05-02
2,Љубов во Рим,Сара Адамс,490.0,0.0,False,Романси,2025-05-02
3,Шпанска љубовна измама,Елена Армас,590.0,0.0,False,Романси,2025-05-02
4,Изопачени лаги,Ана Хуанг,590.0,0.0,False,Романси,2025-05-02
...,...,...,...,...,...,...,...
2161,Аурис,Винсент Клиш,450.0,0.0,False,Трилери,2025-05-02
2162,Кралицата Шарлота,Џулија Квин,520.0,0.0,False,Драми,2025-05-02
2163,Кралицата Шарлота,Шонда Рајмс,520.0,0.0,False,Драми,2025-05-02
2164,Аурис,Себастијан Фицек,450.0,0.0,False,Драми,2025-05-02
